<a href="https://colab.research.google.com/github/sherryjasal/ASR/blob/main/Fine_Tune_XLSR_Wav2Vec2_on_Arabic_ASR_with_%F0%9F%A4%97_Transformers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
!pip3 install datasets
!pip3 install torchaudio
!pip3 install librosa
!pip3 install jiwer

In [2]:
!pip3 install git+https://github.com/huggingface/transformers

  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-hs_71q5_
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-hs_71q5_
  Resolved https://github.com/huggingface/transformers to commit faf25c040d191cee9bd4d3b1d4fd6974f7fd3516
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [3]:
from datasets import load_dataset, load_metric, Audio

common_voice_train = load_dataset("common_voice", "ar", split="train+validation")
common_voice_test = load_dataset("common_voice", "ar", split="test")

/root/.cache/huggingface/modules/datasets_modules/datasets/common_voice/220833898d6a60c50f621126e51fb22eb2dfe5244392c70dccd8e6e2f055f4bf/common_voice.py:634: FutureWarning: 
            This version of the Common Voice dataset is deprecated.
            You can download the latest one with
            >>> load_dataset("mozilla-foundation/common_voice_11_0", "en")
            
  warnings.warn(
/root/.cache/huggingface/modules/datasets_modules/datasets/common_voice/220833898d6a60c50f621126e51fb22eb2dfe5244392c70dccd8e6e2f055f4bf/common_voice.py:634: FutureWarning: 
            This version of the Common Voice dataset is deprecated.
            You can download the latest one with
            >>> load_dataset("mozilla-foundation/common_voice_11_0", "en")
            
  warnings.warn(


In [4]:
common_voice_train

Dataset({
    features: ['client_id', 'path', 'audio', 'sentence', 'up_votes', 'down_votes', 'age', 'gender', 'accent', 'locale', 'segment'],
    num_rows: 21744
})

In [5]:
common_voice_train[10]

{'client_id': '3ed0ada6e07c029a8e33c022429b902ecda77dbb159e19fc06af5af7efb590081b2105a87c4eecf0a68785a396bcd4590fcc175ad7e995a94182ff6b13d26eb7',
 'path': '/root/.cache/huggingface/datasets/downloads/extracted/d683d02b419074b255de0b64800dd56845f714dbe57db09390b0ee84f10622e7/cv-corpus-6.1-2020-12-11/ar/clips/common_voice_ar_19237901.mp3',
 'audio': {'path': '/root/.cache/huggingface/datasets/downloads/extracted/d683d02b419074b255de0b64800dd56845f714dbe57db09390b0ee84f10622e7/cv-corpus-6.1-2020-12-11/ar/clips/common_voice_ar_19237901.mp3',
  'array': array([ 0.00000000e+00,  2.47329623e-14,  4.34723736e-14, ...,
          8.93199257e-08, -2.00373734e-07, -4.29436739e-07]),
  'sampling_rate': 48000},
 'sentence': 'اخرج!',
 'up_votes': 2,
 'down_votes': 0,
 'age': 'twenties',
 'gender': 'male',
 'accent': '',
 'locale': 'ar',
 'segment': "''"}

In [6]:
common_voice_train = common_voice_train.remove_columns(["accent", "age", "client_id", "down_votes", "gender", "locale", "segment", "up_votes"])
common_voice_test = common_voice_test.remove_columns(["accent", "age", "client_id", "down_votes", "gender", "locale", "segment", "up_votes"])

In [7]:
from datasets import ClassLabel
import random
import pandas as pd
from IPython.display import display, HTML

def view_random_samples(dataset, num_samples=10):
    assert num_samples <= len(dataset), "Can't pick more elements than there are in the dataset."
    examples = []
    for _ in range(num_samples):
        example = random.randint(0, len(dataset)-1)
        while example in examples:
            example = random.randint(0, len(dataset)-1)
        examples.append(example)

    df = pd.DataFrame(dataset[examples])
    display(HTML(df.to_html()))

In [8]:
view_random_samples(common_voice_train.remove_columns(["path", "audio"]), num_samples=10)

,sentence
0,أود أن تقابل صديقاً لي.
1,ثمة كتب كثيرة في غرفته.
2,ويستعجلونك بالعذاب ولولا أجل مسمى لجاءهم العذاب وليأتينهم بغتة وهم لا يشعرون
3,.تعرف أنّ جون كان يحبها
4,كان سامي يعمل لدى صاحب بيته السابق.
5,وَإِنْ كَانَ الْعَقْدُ رَغْبَةً فِي الْأُلْفَةِ فَهَذَا يَكُونُ عَلَى أَحَدِ وَجْهَيْنِ
6,انها لا تسمعه.
7,أشعر بآلام في معدتي بعد الأكل.
8,لقد إستيقظتُ متأخرةً اليوم.
9,أنا ما شربت الماء.


In [9]:
import re
ignore_regex = '[\,\?\.\!\-\;\:\"\“\%\‘\”\�]'

def delete_special_chars(batch):
    batch["sentence"] = re.sub(ignore_regex, '', batch["sentence"]).lower() + " "
    return batch

In [10]:
common_voice_train = common_voice_train.map(delete_special_chars)
common_voice_test = common_voice_test.map(delete_special_chars)

In [11]:
view_random_samples(common_voice_train.remove_columns(["path","audio"]))

,sentence
0,ماذا تعتقدون ؟
1,ولا يسأل حميم حميما
2,بدأت أغضب
3,قسمتنا مدرستنا إلى مجموعتين
4,أنت لا تستسلم بسهولة جداً ، أليس كذلك ؟
5,فَلَا يَخْلُو حَالَ تَقْصِيرِهِ مِنْ أَرْبَعَةِ أَحْوَالٍ
6,والنجم والشجر يسجدان
7,يَسْتَبْشِرُونَ بِنِعْمَةٍ مِنَ اللَّهِ وَفَضْلٍ وَأَنَّ اللَّهَ لَا يُضِيعُ أَجْرَ الْمُؤْمِنِينَ
8,وَتَرْكَ التَّسَلُّطِ أَعَطْفُ عَلَى الْمَحَبَّةِ
9,حَتَّى يَصِيرَ قَاهِرًا لِلسَّرَائِرِ


In [12]:
def extract_all_characters(batch):
  entire_text = " ".join(batch["sentence"])
  vocabulary = list(set(entire_text))
  return {"vocab": [vocabulary], "entire_text": [entire_text]}

In [13]:
vocab_train = common_voice_train.map(extract_all_characters, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=common_voice_train.column_names)
vocab_test = common_voice_test.map(extract_all_characters, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=common_voice_test.column_names)

Map:   0%|          | 0/21744 [00:00<?, ? examples/s]

Map:   0%|          | 0/7622 [00:00<?, ? examples/s]

In [14]:
vocabulary_list = list(set(vocab_train["vocab"][0]) | set(vocab_test["vocab"][0]))

In [15]:
vocabulary_dict = {v: k for k, v in enumerate(vocabulary_list)}
vocabulary_dict

{' ': 0,
 'ف': 1,
 'ٌ': 2,
 'ئ': 3,
 'و': 4,
 'ً': 5,
 'ی': 6,
 'ي': 7,
 'ۖ': 8,
 '_': 9,
 'ص': 10,
 'ج': 11,
 'ْ': 12,
 'ء': 13,
 'ب': 14,
 'ِ': 15,
 'ٰ': 16,
 'ّ': 17,
 'ھ': 18,
 'د': 19,
 'ذ': 20,
 'إ': 21,
 'ک': 22,
 'ض': 23,
 'ه': 24,
 '»': 25,
 'e': 26,
 'ط': 27,
 'أ': 28,
 'ُ': 29,
 'ن': 30,
 '؟': 31,
 'ﻻ': 32,
 'َ': 33,
 'ش': 34,
 'غ': 35,
 'چ': 36,
 'ح': 37,
 '—': 38,
 'ـ': 39,
 '،': 40,
 'g': 41,
 'ت': 42,
 'م': 43,
 'ى': 44,
 'ل': 45,
 'ك': 46,
 'ز': 47,
 'ع': 48,
 'ڨ': 49,
 'ة': 50,
 'ﺃ': 51,
 '؛': 52,
 'ٍ': 53,
 't': 54,
 'ا': 55,
 '«': 56,
 'ظ': 57,
 'ؤ': 58,
 'ق': 59,
 'خ': 60,
 'س': 61,
 '☭': 62,
 'ۚ': 63,
 'ث': 64,
 'ر': 65,
 'آ': 66}

In [16]:
vocabulary_dict["[UNK]"] = len(vocabulary_dict)
vocabulary_dict["[PAD]"] = len(vocabulary_dict)
len(vocabulary_dict)

69

In [17]:
import json
with open('vocab.json', 'w') as vocabulary_file:
    json.dump(vocabulary_dict, vocabulary_file)

In [18]:
from transformers import Wav2Vec2CTCTokenizer

tokenizer = Wav2Vec2CTCTokenizer("./vocab.json", unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|")

In [19]:
from transformers import Wav2Vec2FeatureExtractor

feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1, sampling_rate=16000, padding_value=0.0, do_normalize=True, return_attention_mask=True)

In [20]:
from transformers import Wav2Vec2Processor

processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)

In [21]:
from google.colab import drive
drive.mount('/content/gdrive/')

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).


In [22]:
processor.save_pretrained("/content/gdrive/MyDrive/wav2vec2-large-xlsr-arabic-demo")

In [23]:
common_voice_train[0]["path"]

'/root/.cache/huggingface/datasets/downloads/extracted/d683d02b419074b255de0b64800dd56845f714dbe57db09390b0ee84f10622e7/cv-corpus-6.1-2020-12-11/ar/clips/common_voice_ar_19225971.mp3'

In [24]:
common_voice_train[0]["audio"]

{'path': '/root/.cache/huggingface/datasets/downloads/extracted/d683d02b419074b255de0b64800dd56845f714dbe57db09390b0ee84f10622e7/cv-corpus-6.1-2020-12-11/ar/clips/common_voice_ar_19225971.mp3',
 'array': array([ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
         2.36669189e-06, -2.57932743e-06,  6.87623424e-06]),
 'sampling_rate': 48000}

In [25]:
common_voice_train = common_voice_train.cast_column("audio", Audio(sampling_rate=16_000))
common_voice_test = common_voice_test.cast_column("audio", Audio(sampling_rate=16_000))

In [26]:
common_voice_train[0]["audio"]

{'path': '/root/.cache/huggingface/datasets/downloads/extracted/d683d02b419074b255de0b64800dd56845f714dbe57db09390b0ee84f10622e7/cv-corpus-6.1-2020-12-11/ar/clips/common_voice_ar_19225971.mp3',
 'array': array([0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        2.71172757e-06, 9.98028008e-06, 5.87778959e-06]),
 'sampling_rate': 16000}

In [27]:
import IPython.display as ipd
import numpy as np
import random

rand_int = random.randint(0, len(common_voice_train)-1)

ipd.Audio(data=common_voice_train[rand_int]["audio"]["array"], autoplay=True, rate=16000)

In [28]:
rand_int = random.randint(0, len(common_voice_train)-1)

print("Target text:", common_voice_train[rand_int]["sentence"])
print("Input array shape:", common_voice_train[rand_int]["audio"]["array"].shape)
print("Sampling rate:", common_voice_train[rand_int]["audio"]["sampling_rate"])

Target text: اليابان هي جزء آسيا الشرقي 
Input array shape: (63360,)
Sampling rate: 16000


In [29]:
def prepare_dataset(batch):
    audio = batch["audio"]

    # batched output is "un-batched"
    batch["input_values"] = processor(audio["array"], sampling_rate=audio["sampling_rate"]).input_values[0]

    with processor.as_target_processor():
        batch["labels"] = processor(batch["sentence"]).input_ids
    return batch

In [30]:
common_voice_train = common_voice_train.map(prepare_dataset, remove_columns=common_voice_train.column_names, num_proc=4)
common_voice_test = common_voice_test.map(prepare_dataset, remove_columns=common_voice_test.column_names, num_proc=4)

Map (num_proc=4):   0%|          | 0/21744 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:155: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:155: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:155: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call

Map (num_proc=4):   0%|          | 0/7622 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:155: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:155: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:155: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call

In [31]:
import torch

from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union

@dataclass
class DataCollatorCTCWithPadding:
    """
    Data collator that will dynamically pad the inputs received.
    Args:
        processor (:class:`~transformers.Wav2Vec2Processor`)
            The processor used for proccessing the data.
        padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
            Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
            among:
            * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
              sequence if provided).
            * :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
              maximum acceptable input length for the model if that argument is not provided.
            * :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
              different lengths).
        max_length (:obj:`int`, `optional`):
            Maximum length of the ``input_values`` of the returned list and optionally padding length (see above).
        max_length_labels (:obj:`int`, `optional`):
            Maximum length of the ``labels`` returned list and optionally padding length (see above).
        pad_to_multiple_of (:obj:`int`, `optional`):
            If set will pad the sequence to a multiple of the provided value.
            This is especially useful to enable the use of Tensor Cores on NVIDIA hardware with compute capability >=
            7.5 (Volta).
    """

    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True
    max_length: Optional[int] = None
    max_length_labels: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None
    pad_to_multiple_of_labels: Optional[int] = None

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lenghts and need
        # different padding methods
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                max_length=self.max_length_labels,
                pad_to_multiple_of=self.pad_to_multiple_of_labels,
                return_tensors="pt",
            )

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels

        return batch

In [32]:
data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)

In [33]:
wer_metric = load_metric("wer")

<ipython-input-33-605de035dfd7>:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  wer_metric = load_metric("wer")


In [34]:
def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids)
    # we do not want to group tokens when computing the metrics
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)

    wer = wer_metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

In [35]:
from transformers import Wav2Vec2ForCTC

model = Wav2Vec2ForCTC.from_pretrained(
    "facebook/wav2vec2-large-xlsr-53",
    attention_dropout=0.1,
    hidden_dropout=0.1,
    feat_proj_dropout=0.0,
    mask_time_prob=0.05,
    layerdrop=0.1,
    ctc_loss_reduction="mean",
    pad_token_id=processor.tokenizer.pad_token_id,
    vocab_size=len(processor.tokenizer)
)

Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-large-xlsr-53 and are newly initialized: ['lm_head.weight', 'lm_head.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [36]:
model.freeze_feature_extractor()

/usr/local/lib/python3.10/dist-packages/transformers/models/wav2vec2/modeling_wav2vec2.py:1914: FutureWarning: The method `freeze_feature_extractor` is deprecated and will be removed in Transformers v5.Please use the equivalent `freeze_feature_encoder` method instead.
  warnings.warn(


In [37]:
model.gradient_checkpointing_enable()

In [38]:
#!pip3 install transformers[torch]

In [39]:
#!pip3 install accelerate>=0.20.1 -U

In [40]:
!pip3 install git+https://github.com/huggingface/accelerate

  Cloning https://github.com/huggingface/accelerate to /tmp/pip-req-build-g_apmkc6
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/accelerate /tmp/pip-req-build-g_apmkc6
  Resolved https://github.com/huggingface/accelerate to commit 0b36ca6e64245b123e9455521c1bd985f4b72679
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [41]:
import torch
from transformers import TrainingArguments

In [42]:
training_args = TrainingArguments(
  output_dir="/content/gdrive/MyDrive/wav2vec2-large-xlsr-arabic-demo",
  group_by_length=True,
  per_device_train_batch_size=16,
  gradient_accumulation_steps=2,
  evaluation_strategy="steps",
  num_train_epochs=30,
  save_steps=100,
  eval_steps=100,
  logging_steps=10,
  learning_rate=3e-4,
  warmup_steps=500,
  save_total_limit=2,
)

In [43]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=common_voice_train,
    eval_dataset=common_voice_test,
    tokenizer=processor.feature_extractor,
)

In [44]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:155: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(


Step,Training Loss,Validation Loss,Wer
100,7.641000,5.696074,1.000000
200,3.484400,3.419553,1.000000
300,3.428800,3.395977,1.000000


/usr/local/lib/python3.10/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:155: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:155: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:155: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call

Step,Training Loss,Validation Loss,Wer
100,7.641000,5.696074,1.000000
200,3.484400,3.419553,1.000000
300,3.428800,3.395977,1.000000
400,3.439100,3.345433,1.000000


KeyboardInterrupt: ignored